In [1]:
import tensorflow as tf
import keras
import matplotlib.pyplot as plt

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

print("[C1: ln9]: Modules Imported successfully")

[C1: ln9]: Modules Imported successfully


In [2]:
# root Folders
test_root = "dataset/test"
train_root = "dataset/train"
valid = "dataset/valid"

# test
test_adenocarcinoma = "dataset/test/adenocarcinoma"
test_lc_carcinoma = "dataset/test/large.cell.carcinoma"
test_normal = "dataset/test/normal"
test_sc_carcinoma = "dataset/test/squamous.cell.carcinoma"

#train
train_adenocarcinoma = "dataset/train/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib"
train_lc_carcinoma = "dataset/train/large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa"
train_normal = "dataset/train/normal"
train_sc_carcinoma = "dataset/train/squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa"

#validation
valid_adenocarcinoma = "dataset/valid/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib"
valid_lc_carcinoma = "dataset/valid/large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa"
valid_normal = "dataset/valid/normal"
valid_sc_carcinoma = "dataset/valid/squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa"

print("[C2: ln24]:  Data files initialized successfully.\n")

print("[C2: ln26]: Summary\n")
print(f"Type          \t\t Test     \t\t Train     \t\t Validation\n"
      f"Adenocarcinoma\t\t {len(test_adenocarcinoma)}     \t\t {len(train_adenocarcinoma)}     \t\t {len(valid_adenocarcinoma)}\n"
      f"LC_carcinoma  \t\t {len(test_lc_carcinoma)}       \t\t {len(train_lc_carcinoma)}     \t\t {len(valid_lc_carcinoma)}\n"
      f"SC_carcinoma  \t\t {len(test_sc_carcinoma)}       \t\t {len(train_sc_carcinoma)}     \t\t {len(valid_sc_carcinoma)}\n"
      f"Normal        \t\t {len(test_normal)}     \t\t {len(train_normal)}     \t\t {len(valid_normal)}\n"
      f"++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n")

[C2: ln24]:  Data files initialized successfully.

[C2: ln26]: Summary

Type          		 Test     		 Train     		 Validation
Adenocarcinoma		 27     		 56     		 56
LC_carcinoma  		 33       		 59     		 59
SC_carcinoma  		 36       		 62     		 62
Normal        		 19     		 20     		 20
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++



In [3]:
print("[C3: ln1]: Data Preparation Initiated")

batch_size = 2
img_h, img_w = 255,255

test_datagen = ImageDataGenerator(
      rescale=1./255,
      #target_size=(255,255),
      horizontal_flip=True,
      brightness_range=(0,7),
    rotation_range=40,
    shear_range=0.2,
    zoom_range = 0.2,
    fill_mode = 'nearest'
    
)


train_datagen =ImageDataGenerator(
      rescale=1./255,
      horizontal_flip=True,
      brightness_range=(0,7),
    rotation_range=40,
    shear_range=0.2,
    zoom_range = 0.2,
    fill_mode = 'nearest'
)

train_generator = train_datagen.flow_from_directory(
      train_root,
     target_size=(255,255),
      batch_size=batch_size
)

valid_generator = test_datagen.flow_from_directory(
      test_root,
      batch_size = batch_size,
    target_size=(255,255)
)

print("[C3: ln30]: Data Preparation Completed.\n")

[C3: ln1]: Data Preparation Initiated
Found 613 images belonging to 4 classes.
Found 315 images belonging to 4 classes.
[C3: ln30]: Data Preparation Completed.



In [4]:
# creating callbacks
def schedule(epoch, lr):
    if epoch > 50:
        return lr * 0.5
    else:
        return lr
        
learning_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(schedule=schedule)
reduce_lr = ReduceLROnPlateau( monitor="val_loss", factor=0.2, patience= 5, min_lr=0.001)
early_stop = EarlyStopping(monitor="val_loss",patience=50,min_delta=0.0001)
model_checkpoint = ModelCheckpoint("updated_model.keras",monitor="val_loss",save_best_only=True)

print("[C4: ln7]: callbacks Created successfully.")

[C4: ln7]: callbacks Created successfully.


In [5]:
print("[C5: ln1]: Building ResNet Model")

resNet_model = Sequential()
pretrained_model = ResNet50(
      include_top = False, input_shape = (img_h, img_w, 3), pooling='avg', weights='imagenet'
)

for layer in pretrained_model.layers:
      layer.trainable = False


resNet_model.add(pretrained_model)
resNet_model.add(layers.Flatten())
resNet_model.add(layers.Dense(77, activation="relu"))
resNet_model.add(layers.Dense(32, activation="relu"))
resNet_model.add(layers.Dense(4, activation="softmax"))



[C5: ln1]: Building ResNet Model


In [ ]:

print("[C6: ln1]:  Training Model")
resNet_model.compile(optimizer= tf.keras.optimizers.Adam(0.1), loss= "categorical_crossentropy", metrics=['accuracy'])
history = resNet_model.fit(
    train_generator,
    validation_data = valid_generator,
    epochs=150,
    callbacks=[reduce_lr, early_stop, model_checkpoint, learning_rate_scheduler],
)

print("[C6: ln5]: Model Training Successfull")

[C6: ln1]:  Training Model
Epoch 1/150


C:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


307/307 ━━━━━━━━━━━━━━━━━━━━ 346s 1s/step - accuracy: 0.2847 - loss: 6.4210 - val_accuracy: 0.3810 - val_loss: 1.3363 - learning_rate: 0.1000
Epoch 2/150
307/307 ━━━━━━━━━━━━━━━━━━━━ 333s 1s/step - accuracy: 0.2928 - loss: 1.4025 - val_accuracy: 0.3810 - val_loss: 1.3377 - learning_rate: 0.1000
Epoch 3/150
307/307 ━━━━━━━━━━━━━━━━━━━━ 628s 2s/step - accuracy: 0.2665 - loss: 1.4028 - val_accuracy: 0.2857 - val_loss: 1.3422 - learning_rate: 0.1000
Epoch 4/150
307/307 ━━━━━━━━━━━━━━━━━━━━ 485s 2s/step - accuracy: 0.3149 - loss: 1.3897 - val_accuracy: 0.3810 - val_loss: 1.3602 - learning_rate: 0.1000
Epoch 5/150
307/307 ━━━━━━━━━━━━━━━━━━━━ 574s 2s/step - accuracy: 0.3674 - loss: 1.3684 - val_accuracy: 0.3810 - val_loss: 1.3634 - learning_rate: 0.1000
Epoch 6/150
307/307 ━━━━━━━━━━━━━━━━━━━━ 480s 2s/step - accuracy: 0.2965 - loss: 1.3914 - val_accuracy: 0.2857 - val_loss: 1.4620 - learning_rate: 0.0200
Epoch 7/150
307/307 ━━━━━━━━━━━━━━━━━━━━ 536s 2s/step - accuracy: 0.3107 - loss: 1.3852 

In [ ]:
%matplotlib inline
loss_plt = history.history['loss']
accuracy_plt = history.history['accuracy']
vAccuracy_plt = history.history['val_accuracy']
vLoss_plt = history.history['val_loss']
lr_plt = history.history['learning_rate']

print("[C7: ln5]: Model Results Obtained")

In [ ]:
plt.plot(loss_plt, label="Loss")
plt.plot(vLoss_plt, label="Val Loss")
plt.title("Loss vs Epoch")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
print("[C8: ln4]: Plotted LOSS vs EPOCH")

In [ ]:
plt.plot(accuracy_plt, label="Accuracy")
plt.plot(vAccuracy_plt, label="Val Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Accuracy vs Epoch")
plt.legend()

print("[C9: ln4]: Plotted ACCURACY vs EPOCH")

In [ ]:
plt.plot(lr_plt)
plt.title("Learning Rate vs Epoch")
plt.xlabel("Epoch")
plt.ylabel("Learning Rate")

print("[C9: ln4]: Plotted Learning Rate vs EPOCH")

In [ ]:
# Fetching the necessary data
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

train_loss = history.history['loss']
val_loss = history.history['val_loss']

lr = history.history['learning_rate']

#print(f"{train_accuracy}\n\n{val_accuracy}\n\n{train_loss}\n\n{val_loss}\n\n")

file_1 = open(r"resNet Results\Values\train_accuracy.txt",'w')
file_2 = open(r"resNet Results\Values\val_accuracy.txt",'w')
file_3 = open(r"resNet Results\Values\train_loss.txt",'w')
file_4 = open(r"resNet Results\Values\val_loss.txt",'w')
file_5 = open(r"resNet Results\Values\learning_rate.txt",'w')

for i in train_accuracy:
    i = str(i)+"\n"
    file_1.write(i)
    
print("[C11: ln15]: Train_accuracy data appended_successfully")
file_1.close()

for i in val_accuracy:
    i = str(i)+"\n"
    file_2.write(i)
    
print("[C11: ln21]: Val_accuracy data appended_successfully")
file_2.close()

for i in train_loss:
    i = str(i)+"\n"
    file_3.write(i)
    
print("[C11: ln26]: Train_loss data appended_successfully")
file_3.close()

for i in val_loss:
    i = str(i)+"\n"
    file_4.write(i)
    
print("[C11: ln31]: Val_loss data appended_successfully")
file_4.close()

for i in lr:
    i = str(i)+"\n"
    file_5.write(i)
    
print("[C11: ln50]: Learning Rate data appended_successfully")
file_5.close()

EPOCH : 04:40 AM Started 05:52AM Completed
EPOCH : 06:29 AM Started 07:34AM Completed